##**Trabalho Prático 1**
## Bárbara  Faria (A85774) , Bruna Araújo (A84408) e Tiago Lima (A85126)

## 1.Horário Semanal 

> O propósito deste trabalho é a análise de problemas de alocação usando técnicas de SAT,  em lógica proposicional, e IP em lógica linear inteira.


1. Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:

    1. Cada reunião ocupa uma sala (enumeradas 1..S) durante um “slot” (tempo,dia).  Assume-se os dias enumerados 1..D e, em cada dia, os tempos enumerados 1..T.
    2.  Cada reunião tem associado um projeto (enumerados 1..P) e um conjunto de participantes. Os diferentes colaboradores são enumerados 1..C.
    3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. São “inputs” do problema o conjunto de colaboradores de cada projeto, o seu líder e o número de reuniões semanais.
    4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  50\% do total de colaboradores do projeto.  A disponibilidade de cada participante, incluindo o lider,  é um conjunto de “slots” (“inputs” do problema). 

In [ ]:
!pip install ortools

### Análise do problema

Existem $C$ colaboradores ,identificados por um índice $c \in [0..C\!-\!1]$, que pertence a um determinado projeto $p \in [0..P\!-\!1]$, , e podemos identificar cada sala disponível num dado dia, num certo tempo, por um triplo $(s,d,t)\in[0..S\!-\!1]\times[0..D\!-\!1]\times[0..T\!-\!1]$.

Vamos usar uma família $X_{c,p,s,d,t}$ de variáveis binárias (i.e., que assumem valores inteiros $\{0,1\}$), com a seguinte semântica

$$X_{p,c,s,d,t} == 1  \quad \mbox{se e só se} \quad \mbox{a reunião do projeto $p$ com $c$ colaboradores está localizada na sala $s$ no dia $d$ no tempo $t$} $$

Para facilitar a contagem do numero de reuniões vamos usar uma família $K_{p,s,d,t}$ de variáveis binárias, com a seguinte semântica

$$K_{p,s,d,t} == 1  \quad \mbox{se e só se} \quad \mbox{A reunião do projeto $p$ está localizada na sala $s$ no dia $d$ no tempo $t$} $$


Para representar os líderes de cada projeto uma família $Z_{c,p}$ de variáveis binárias, com a seguinte semântica

$$Y_{p,c} == 1  \quad \mbox{se e só se} \quad \mbox{O projeto p contém c colaboradores} $$


Destaca-se ainda o seguinte:

**Limitações:** (que impõem limites máximos à alocação)
1. Cada sala tem alocada no maximo 1 projeto.
2. Um colaborador não pode frequentar duas reuniões ao mesmo tempo.




**Obrigações:** (que impõem limites mínimos à alocação)
1. Cada projeto realiza um dado número de reuniões semanais, em que o minímo será uma e no máximo N.
2. O líder tem de estar obrigatoriamente na reunião do projeto que lidera.
3.Cada reunião tem de ter presentes pelo menos 50% dos colaboradores.









### Implementação

Começamos por importar a biblioteca de programação linear do OR-Tools e criar uma instância do *solver*.


Depois inicializamos o *solver* `horario` e definir os valores para as constantes $S$, $T$, $D$, $P$, $C$ e $N$.

In [ ]:
from ortools.linear_solver import pywraplp

horario = pywraplp.Solver.CreateSolver('SCIP')

S, P, D, T, C, N = 1, 2, 5, 6, 21, 4

Declaração das matrizes de alocação:

In [ ]:
x={}
for p in range(P):
  x[p] = {}
  for c in range(C):
    x[p][c] = {}
    for s in range(S):
     x[p][c][s] = {}
     for d in range(D):
      x[p][c][s][d] = {}
      for t in range(T):
        x[p][c][s][d][t]=horario.BoolVar('x[%i][%i][%i][%i][%i]' %(p,c,s,d,t))

k={}
for p in range(P):
  k[p]={}
  for s in range(S):
    k[p][s]={}
    for d in range(D):
      k[p][s][d]={}
      for t in range(T):
        k[p][s][d][t]=horario.BoolVar('k[%i][%i][%i][%i]' %(p,s,d,t))

y = {}
for p in range(P):
  y[p] = {}
  for c in range(C):
    y[p][c] = horario.BoolVar('y[%i][%i]' % (p,c))
    
                
def X(p,c,s,d,t):
    return x[p][c][s][d][t]

def K(p,s,d,t):
    return k[p][s][d][t]

def Y(p,c):
    return y[p][c]

Passamos agora á modelação das restrições.

#### Limitações:

1.Cada slot tem alocada no máximo uma reunião.

   Vamos percorrer todas as salas em todos os dias e tempos e calculamos o somatório de todos os projetos  e esse somatório tem de ser menor ou igual a 1 de modo a que cada sala tem 0 ou 1 projetos associados. Esta restrição é feita no K pois não temos de nos preocupar com os colaboradores.


$$\forall{s<S} \cdot \forall{d<D} \cdot \forall{t<T} \cdot \quad \sum_{p<P} k_{p,s,d,t} \leq 1$$

In [ ]:
for s in range(S):
 for d in range(D):
  for t in range(T):
      horario.Add(sum([K(p,s,d,t) for p in range(P)]) <= 1)

2.Um colaborador não pode frequentar duas reuniões ao mesmo tempo.

Vamos percorrer todos os colaboradores em todos os dias e todos os tempos e para cada combinação ($c$,$d$,$t$) calculamos o somatório de todos as salas e projetos (reuniões) e esse valor tem de ser igual ou inferior a 1, garantido assim que um colaborador num terminado dia num determinado tempo está no máximo em uma reunião.


$$\forall{c<C} \cdot \forall{d<D} \cdot \forall{t<T} \cdot \quad \sum_{p<P, s<S} x_{p,c,s,d,t} \leq 1$$

In [ ]:
for c in range(C):
  for d in range(D):
     for t in range(T):
       horario.Add(sum([X(p,c,s,d,t) for p in range(P) for s in range(S)]) <= 1)


#### Obrigações:

1. Cada projeto realiza um dado número de reuniões semanais, em que o minímo será uma e no máximo N.

  Percorremos cada projeto e para cada um calculamos o somatório de todas as salas, dias e tempos e esse valor tem de estar compreendido entre 1 e N.


$$\forall{p<P}  \cdot \quad 1 \leq \sum_{s<S d<D, t<T} k_{p,s,d,t} \leq N$$

In [ ]:
for p in range(P):
  horario.Add(sum([X(p,c,s,d,t) for s in range(S) for d in range(D) for t in range(T)]) <= N)
  horario.Add(sum([X(p,c,s,d,t) for s in range(S) for d in range(D) for t in range(T)]) >= 1)

2. Cada projeto tem de ter um lider.

  Percorremos todos os projetos e a restrição é feita do Y , onde fazemos o somatório de todos os colaboradores e se este for maior ou igual que 1 é garantido que existe líder. 

  
$$\forall{p<P}  \cdot \quad \sum_{c<C} y_{p,c} \geq 1$$

In [ ]:
for p in range(P):
     horario.Add(sum([Y(p,c) for c in range(C)]) >= 1)

 3. A capacidade minima em cada reuniao é de 50% dos colaboradores desse mesmo projeto.

 $$\forall{p<P}  \cdot \forall{s<S} \cdot \forall{d<D} \cdot \forall{t<T} \cdot\quad \sum_{c<C} x_{p,c,s,d,t}  \geq C//2 $$

In [ ]:
for p in range(P):
  for s in range(S):
    for d in range(D):
      for t in range(T):
        horario.Add((sum([X(p,c,s,d,t) for c in range(C)])) >= C//2)

###Exemplo de impressão de um horário



In [ ]:
status=horario.Solve()
if status==pywraplp.Solver.OPTIMAL:
  #imprime horario de todas as reuniões
  for p in range(P):
    print("Projeto:",p,"\n")
    for d in range(D):
      for t in range(T):
        for s in range(S):
#         print(X(p,c,s,d,t).solution_value())
          if round(X(p,c,s,d,t).solution_value())==1:
            print("Dia",d,"-> Tempo",t, "->","Sala:",s,"\n")
    print("Colaboradores:")
    for c in range(C):
      if round(X(p,c,s,d,t).solution_value()) == 1: 
        print(" ->",c,end='')
    print("\n")
else:
  print("Sem solução!")

Projeto: 0 

Dia 0 -> Tempo 0 -> Sala: 0 

Dia 0 -> Tempo 1 -> Sala: 0 

Dia 0 -> Tempo 2 -> Sala: 0 

Dia 0 -> Tempo 3 -> Sala: 0 

Colaboradores:
 -> 0 -> 11 -> 12 -> 13 -> 14 -> 15 -> 16 -> 17 -> 18 -> 19

Projeto: 1 

Dia 0 -> Tempo 4 -> Sala: 0 

Dia 0 -> Tempo 5 -> Sala: 0 

Dia 1 -> Tempo 0 -> Sala: 0 

Dia 1 -> Tempo 1 -> Sala: 0 

Colaboradores:
 -> 1 -> 2 -> 3 -> 4 -> 5 -> 6 -> 7 -> 8 -> 9 -> 10

